# RAG simple PDF


L'objectif de ce notebook est d'expérimenter un RAG simple sur le fichier pdf du code civil Ivoirien, avec le modèle Gemma2 en local.



### SOMMAIRE  
[1. Prepare data (load and chunk)](#prepare-data-load-chunk)  
[2. Vector database](#vector-database)  
[3. RAG](#rag)  

##### Packages requirements list

In [1]:
!pip freeze

aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
annotated-types==0.7.0
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==24.2.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.5
decorator==5.1.1
defusedxml==0.7.1
dnspython==2.6.1
docarray==0.40.0
email_validator==2.2.0
executing==2.1.0
fastapi==0.115.0
fastapi-cli==0.0.5
fastjsonschema==2.20.0
filelock==3.16.0
fqdn==1.5.1
frozenlist==1.4.1
fsspec==2024.9.0
greenlet==3.0.3
h11==0.14.0
httpcore==1.0.5
httptools==0.6.1
httpx==0.27.2
huggingface-hub==0.24.7
idna==3.8
ipykernel==6.29.5
ipython==8.27.0
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.4
joblib==1.4.2
json5==0.9.25
jsonpatch==1.33
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2023.12.1
jupyter==1.1.1
jupyter-console==6.6.3
jupyter-ev

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
simple_logger = logging.getLogger(__name__)

### <a id='prepare-data-load-chunk'>1. Prepare data (load and chunk)</a>

In [2]:
model_name = 'gemma2:2b-instruct-q4_1' # 'gemma2:9b-instruct-q4_1'

In [3]:
# --- Charger le fichier
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

fichier = Path('../data/pdf/rag-civ/211.10.64-Code-civil-I-ivoirien.pdf')
loader = PyPDFLoader(fichier)
pages = loader.load_and_split()

In [4]:
# check
pages[1]

Document(metadata={'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf', 'page': 1}, page_content="3 TITRE PRELIMINAIRE : \nDE LA PUBLICATION, DES EFFETS  \nET DE L'APPLICATION DES LOIS EN GENERAL \n \n  \nARTICLE 1 \nLes lois sont exécutoires, dans tout le territoire ivoirien, de la \npromulgation qui en est faite par le Président de la République.  \nElles seront exécutées dans chaque partie de la République,  du moment \noù la promulgation en pourra être connue. \n \nARTICLE 2 \nLa loi ne dispose que pour l'avenir, elle n'a point d'effet rétr oactif. \n \n ARTICLE 3 \nLes lois de police et de sûreté obligent tous ceux qui habitent le ter ritoire. \nLes immeubles, même ceux possédés par des étrangers, sont régis  par la \nloi ivoirienne \nLes lois concernant l'état et la capacité des personnes régisse nt les \nIvoiriens, même résidant en pays étrangers. \n \n ARTICLE 4 \nLe juge qui refusera de juger, sous prétexte du silence, de l'obscur ité ou \nde l'insuffisance

In [5]:
# Splitter le texte du fichier par groupes de 1000 tokens
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=['\n\n', '\n', ' '])
chunks = text_splitter.split_documents(pages)

In [6]:
# check
chunks[12]

Document(metadata={'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf', 'page': 9}, page_content="l'individu présumé absent. \n \n ARTICLE 118 \nLe Procureur de la République enverra, aussitôt qu'ils seront rendus, les \njugements, tant préparatoires que définitifs, au ministère de la Jus tice qui \nles rendra publics. \n \n ARTICLE 119 \nLe jugement de déclaration d'absence ne sera rendu qu’un (1 ) an après le \njugement qui aura ordonné l'enquête.")

### <a id='vector-database'>2. Vector database</a>

Créer les embeddings des chunks de texte et les envoyer au vector store.

In [7]:
from langchain_ollama import OllamaEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_postgres.vectorstores import PGVector

In [8]:
# Initialiser le modèle d'embeddings
# embeddings = OllamaEmbeddings(model=model_name)
embeddings = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    model_name="bert-base-multilingual-cased")

C:\Users\Nelly\AppData\Local\Temp\ipykernel_18820\1110162454.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: ber

In [9]:
# Envoyer les chunks au vector store ou lire l'existant
import psycopg


scheme = "postgresql"
connection = f"{scheme}://raguser:abcdefgh@localhost:5432/testdb"
conn = psycopg.connect(conninfo=connection)
schema_name = "rags"
collection_name = "code-civil-ci"

try:
    scheme = "postgresql+psycopg"
    connection = f"{scheme}://raguser:abcdefgh@localhost:5432/testdb"

    simple_logger.info('Checking if collection {0} exists'.format(collection_name))
    cursor = conn.cursor()
    cursor.execute(f"select name from {schema_name}.langchain_pg_collection where name='{collection_name}';")

    cursor.fetchone()[0]

    vector_store = PGVector.from_existing_index(
        embedding=embeddings,
        collection_name=collection_name,
        connection=connection
        )
    simple_logger.info('OK, instance created for existing collection {0}'.format(collection_name))
except:
    simple_logger.info('Collection {0} does not exist. Creating..'.format(collection_name))
    vector_store = PGVector.from_documents(
        embedding=embeddings,
        documents=chunks,
        collection_name=collection_name,
        connection=connection,
        ids=[id+1 for id in range(len(chunks))],
        use_jsonb=True,
    )
    simple_logger.info('Collection {0} successfully created.'.format(collection_name))

conn.close()



INFO:__main__:Checking if collection code-civil-ci exists
INFO:__main__:Collection code-civil-ci does not exist. Creating..
INFO:__main__:Collection code-civil-ci successfully created.


### <a id='rag'>3. RAG</a>

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

In [11]:
# Initialiser le modèle de chat
llm = ChatOllama(model=model_name, temperature=0)

In [12]:
# Initialiser le retriever

retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 10})

prompt_template = '''You are an assistant for question-answering tasks in french. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise. \
Respond in french.
Question: {question} 
Context: {context} 
Answer:
'''

prompt = ChatPromptTemplate.from_template(prompt_template)

In [13]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Poser des questions à notre pdf

In [14]:
# Trouver le tribunal cométent pour une requête conjointe de divorce
rag_chain.invoke('Quel tribunal territorial a la possibilité de répondre à la requête de divorce en cas de consentement ?')

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


"Je n'ai pas trouvé la réponse à votre question dans les documents fournis. \n"

Ajouter la possibilité au modèle d'inclure la source dans la réponse  


In [15]:
from typing_extensions import Annotated, TypedDict
from typing import List

# Schéma de la réponse
class AnswerWithSources(TypedDict):
    answer: str
    sources: Annotated[
        List[str],
        ...,
        "List of sources (author + year) used to answer the question",
    ]

In [20]:
rag_chain2 = (
    {
        "question": lambda x: x["question"],
        "context": lambda x: x["context"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
# On passe la question au retriever
response_docs = (lambda x: x['question']) | retriever

In [22]:
chain_w_source = RunnablePassthrough.assign(context=response_docs).assign(
    answer=rag_chain2
)

In [23]:
chain_w_source.invoke({'question':'Quel tribunal territorial a la possibilité de répondre à la requête de divorce en cas de consentement ?'})

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'question': 'Quel tribunal territorial a la possibilité de répondre à la requête de divorce en cas de consentement ?',
 'context': [Document(id='222', metadata={'page': 128, 'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf'}, page_content="130 Le juge peut, par décision motivée, refuser l'homologation de la \nConvention s'il constate que celui-ci préserve insuffisamment les in térêts \ndes enfants ou de l'un des époux. Dans cette hypothèse, il ne prononc e \npas le divorce. Cette décision de rejet, ainsi que celles rendu es en \nviolation de dispositions d'ordre public, sont susceptibles d' appel par \ndéclaration au greffe du tribunal dans un délai de trente (30) jours à \ncompter du jour de la notification faite aux parties par le greffie r à la \ndiligence du ministère public. \n \nARTICLE 13 \nExtrait du jugement ou de l'arrêt qui prononce le divorce ou la  séparation \nde corps est inséré, à la diligence du ministère public, dans un  journal \nd'annonces lég

In [24]:
chain_w_source.invoke({'question':'Quels sont les mariages interdits en Côte d''Ivoire?'})

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'question': 'Quels sont les mariages interdits en Côte dIvoire?',
 'context': [Document(id='60', metadata={'page': 37, 'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf'}, page_content='éléments de l’acte lui -même établissent, le cas échéant, après vérification, \nque cet acte est irrégulier, falsifié ou que les faits qui y sont décl arés ne \ncorrespondent pas à la réalité. \nCeux de ces actes qui concernent les Ivoiriens, sont transcrits, soit \nd’office, soit à la demande des intéressés, sur les registres de l ’état civil \nde l’année courante tenus par les agents diplomatiques ou les consul s'),
  Document(id='6', metadata={'page': 4, 'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf'}, page_content="étranger. \n \n ARTICLE 16 \nEn toutes matières, l'étranger qui sera demandeur principal ou inte rvenant \nsera tenu de donner caution pour le paiement des frais et dommages-\nintérêts résultant du procès, à moins qu'il ne possède en Côte d'I

In [25]:
chain_w_source.invoke({'question':'La polygamie est-elle autorisée?'})

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'question': 'La polygamie est-elle autorisée?',
 'context': [Document(id='3', metadata={'page': 2, 'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf'}, page_content="4 ARTICLE 5 \nIl est défendu aux juges de prononcer par voie de disposition gé nérale, et \nréglementaire sur les causes qui leur sont soumises. \n \n ARTICLE 6 \nOn ne peut déroger, par des conventions particulières, aux lois qu i \nintéressent l'ordre public et les bonnes mœurs."),
  Document(id='170', metadata={'page': 100, 'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoirien.pdf'}, page_content='nul ne peut se prévaloir des irrégularités de cet acte. \n  \nARTICLE 43 \nNul ne peut contester la légitimité d’un enfant, dont le père ou la mère  est \ndécédé, les fois que cette légitimité est prouvée par une possessi on d’état \nqui n’est point contredite par l’acte de naissance.'),
  Document(id='297', metadata={'page': 170, 'source': '..\\data\\pdf\\rag-civ\\211.10.64-Code-civil-I-ivoi

In [27]:
#TODO : try a different model for LLM and/or embedding for better retrieval.